In [43]:
import argparse
import os
import time
import random
import torch
import torch.nn as nn
import numpy as np
from utils import *
from model import *
from sliding_window import sliding_window
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score, f1_score, accuracy_score

In [44]:
gpu_id = 2 # index starts from 0

if gpu_id>=2:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)

In [45]:
class Args:
    def __init__(self):
        self.config = 'default1'
        self.dataset = 'opportunity_lc'
        self.model = 'UniTS'
        self.log = 'log'
        self.exp = ''
        self.seed = 0
        self.ratio = 0.2
        self.gpu_id = 0
        self.epochs = 25
        self.lr = 1e-3
        # self.sigma = 0.35
        self.batch_size = 64
        self.save = True #'BCE'
        self.test_only = False
        self.input_size = 24 #256    
        self.input_channel = 113 #45
        self.hheads = 9
        self.SENSOR_AXIS = 3
        
        # self.momentum = 0.1
        # self.c = 0.01
        # self.svmLR = 1e-4
        # self.Ntest = 100
        # self.gpuNo = 2
        # self.cuda_id = 2
        # self.multimodalZ = False
        # self.window_len = 512
        # self.stride_len = 20
        # self.act_list = [1, 2, 3, 4, 5, 6, 7, 12, 13, 16, 17, 24]
        # self.imSize = 64
        # self.sigma = [60, 80]
        # # self.sigma = 0.3
        # self.random_seed = 0
        # self.train_split = 0.8
        
args = Args()
args.num_labels=5

args.log_path = os.path.join(args.log, args.dataset)
if not os.path.exists(args.log_path):
    os.mkdir(args.log_path)
torch.cuda.set_device(args.gpu_id)
args.model_save_path = os.path.join(args.log_path, args.model + '_'+ args.config + '.pt')
#args.model_save_path = '........../UniTS_default.pt'
config = read_config(args.config + '.yaml')


In [46]:
def corrupt1(x, sigma=0.2):
    x=torch.Tensor(x)
    noise = sigma * (torch.randn(x.size()).type_as(x))
    return x + noise

In [47]:
def corrupt2(x, sigma=[6,10]):
        # print(x.shape)
        # current: torch.Size([18944, 1, 27, 171])
        # expected: torch.Size([171, 64, 27])
        #9894, 24, 113
        # time * batch_size * feature
        # lambdas reuse the sigma variable, unpack
        lambda_corr = sigma[0] # lambda for missing data period
        lambda_norm = sigma[1] # lambda for normal data periodß
        # corrupted_x = copy.deepcopy(x)
        
        # failure_mat = np.random.uniform(size = x.shape) < failure_rate
        # num_failures = np.sum(failure_mat)
        # failure_durations = np.random.exponential(scale = duration_scale, size = num_failures).astype(int)
        x = torch.from_numpy(x)
        mask = torch.ones_like(x)
        #print(mask)
        #print(x.shape)
        # failure_id = 0
        # sample_id is the batch size : 64
        # ch_id is the features: 113
        # mask.shape[0]: 171
       # 46495, 24, 113
        # print(mask.shape[0])
        # print(mask.shape[1])
        # print(mask.shape[2])
        for sample_id in range(mask.shape[0]):
            for ch_id in range(mask.shape[1]):  
                ptr = 0
                is_corrupted = False
                while ptr < mask.shape[2]:
                    if is_corrupted:
                        corr_duration = int(np.random.exponential(scale=lambda_corr))
                        #  mask[ptr:min(mask.shape[0], ptr + corr_duration), sample_id, ch_id] = 0
                        mask[sample_id ,ch_id, ptr:min(mask.shape[2], ptr + corr_duration)] = 0
                        ptr = min(mask.shape[2], ptr + corr_duration)
                        is_corrupted = False
                    else:
                        norm_duration = int(np.random.exponential(scale=lambda_norm))
                        ptr = min(mask.shape[2], ptr + norm_duration)
                        is_corrupted = True
        return torch.mul(x, mask)   
        
       

In [48]:
def corrupt3(x, sigma=[0.1, 6,10]):
        # print(x.shape)
        # current: torch.Size([18944, 1, 27, 171])
        # expected: torch.Size([171, 64, 27])
        #9894, 24, 113
        x=torch.Tensor(x)

        noise = sigma[0] * (torch.randn(x.size()).type_as(x))
        x= x+noise
        # time * batch_size * feature
        # lambdas reuse the sigma variable, unpack
        lambda_corr = sigma[1] # lambda for missing data period
        lambda_norm = sigma[2] # lambda for normal data periodß
        # corrupted_x = copy.deepcopy(x)
        
        # failure_mat = np.random.uniform(size = x.shape) < failure_rate
        # num_failures = np.sum(failure_mat)
        # failure_durations = np.random.exponential(scale = duration_scale, size = num_failures).astype(int)
        #x = torch.from_numpy(x)
        mask = torch.ones_like(x)
        #print(mask)
        #print(x.shape)
        # failure_id = 0
        # sample_id is the batch size : 64
        # ch_id is the features: 113
        # mask.shape[0]: 171
       # 46495, 24, 113
        print(mask.shape[0])
        print(mask.shape[1])
        print(mask.shape[2])
        for sample_id in range(mask.shape[0]):
            for ch_id in range(mask.shape[1]):  
                ptr = 0
                is_corrupted = False
                while ptr < mask.shape[2]:
                    if is_corrupted:
                        corr_duration = int(np.random.exponential(scale=lambda_corr))
                        #  mask[ptr:min(mask.shape[0], ptr + corr_duration), sample_id, ch_id] = 0
                        mask[sample_id ,ch_id, ptr:min(mask.shape[2], ptr + corr_duration)] = 0
                        ptr = min(mask.shape[2], ptr + corr_duration)
                        is_corrupted = False
                    else:
                        norm_duration = int(np.random.exponential(scale=lambda_norm))
                        ptr = min(mask.shape[2], ptr + norm_duration)
                        is_corrupted = True
        return torch.mul(x, mask)   
        
       

In [49]:
'''
Commented our parse_args to use in jupyter notebook
'''
# def parse_args():
#     parser = argparse.ArgumentParser(description='train and test')
#     parser.add_argument('--config', default = 'default', type =str) # Read UniTS hyperparameters
#     parser.add_argument('--dataset', default = 'opportunity_lc', type = str,
#                         choices=['opportunity_lc', 'seizure', 'wifi', 'keti'])
#     parser.add_argument('--model', default='UniTS', type=str,
#                         choices=['UniTS', 'THAT', 'RFNet', 'ResNet', 'MaDNN', 'MaCNN', 'LaxCat', 'static'])
#     parser.add_argument('--seed', default=0, type=int)
#     parser.add_argument('--log', default='log', type=str,
#                         help="Log directory")
#     parser.add_argument('--exp', default='', type=str,
#                         choices = ['','noise','missing_data'])
#     parser.add_argument('--ratio', default=0.2, type=float)
#     parser.add_argument('--n_gpu', default=0, type =int)
    
#     parser.add_argument('--epochs', default = 50, type = int)
#     parser.add_argument('--lr', default = 1e-3, type = float)
#     parser.add_argument('--batch_size', default = 64, type = int)

#     parser.add_argument('--save', action = 'store_true')
#     parser.add_argument('--test_only', action = 'store_true')
#     args = parser.parse_args()
#     config = read_config(args.config + '.yaml')
#     if not os.path.exists(args.log):
#         os.mkdir(args.log)
#     args.log_path = os.path.join(args.log, args.dataset)
#     if not os.path.exists(args.log_path):
#         os.mkdir(args.log_path)
#     torch.cuda.set_device(args.n_gpu)

#     if args.dataset == 'opportunity_lc':
#         args.input_size = 256
#         args.input_channel = 45
#         args.hheads = 9
#         args.SENSOR_AXIS = 3
#     elif args.dataset == 'seizure':
#         args.input_channel = 18
#         args.input_size = 256
#         args.hheads = 6
#         args.SENSOR_AXIS = 1
#     elif args.dataset == 'wifi':
#         args.input_channel = 180
#         args.input_size = 256
#         args.batch_size = 16
#         args.hheads = 9
#         args.SENSOR_AXIS = 3
#     elif args.dataset == 'keti':
#         args.input_channel = 4
#         args.input_size = 256
#         args.hheads = 4
#         args.SENSOR_AXIS = 1
#     args.model_save_path = os.path.join(args.log_path, args.model + '_'+ args.config + '.pt')
#     return args, config

# args, config = parse_args()
log = set_up_logging(args, config)
args.log = log

layer_num:	1

window_list:	[7, 16]

stride_list:	[3, 8]

k_list:	[3, 8]

hidden_channel:	48



In [ ]:
#Loading Opportunity Dataset.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00226/OpportunityUCIDataset.zip --no-check-certificate

In [ ]:
import zipfile
with zipfile.ZipFile("............/UniTS-Sensory-Time-Series-Classification/source/OpportunityUCIDataset.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [50]:
#from sliding_window import sliding_window
import pickle as cp
from tensorflow.keras.utils import to_categorical

In [ ]:
!python "................./Copy of preprocess_data.py" -h

In [ ]:
!python "......./Copy of preprocess_data.py" -i ........../UniTS/UniTS-Sensory-Time-Series-Classification/source/OpportunityUCIDataset.zip -o oppChallenge_gestures.data -t locomotion

## UniTS original pre-processing

In [ ]:
# def read_data(args, config):
#     path = os.path.join('../dataset', args.dataset)
#     x_train = np.load(os.path.join(path, 'x_train.npy'))
#     y_train = np.load(os.path.join(path, 'y_train.npy')).astype('int64').tolist()
#     x_test = np.load(os.path.join(path, 'x_test.npy'))
#     y_test = np.load(os.path.join(path, 'y_test.npy')).astype('int64').tolist()
#     np.random.seed(args.seed)

#     if args.exp == 'noise': # Robustness test (noise)
#         for i in range(len(x_train)):
#             for j in range(x_train.shape[2]):
#                 noise = np.random.normal(1,1 , size= x_train[i][:, j].shape)
#                 x_train[i][:, j] = x_train[i][:, j] + noise * args.ratio * np.mean(np.absolute(x_train[i][:, j] ))
#         for i in range(len(x_test)):
#             for j in range(x_test.shape[2]):
#                 noise = np.random.normal(1, 1, size= x_test[i][:, j].shape)
#                 x_test[i][:, j] = x_test[i][:, j] + noise * args.ratio * np.mean(np.absolute(x_test[i][:, j] ))

#     elif args.exp == 'missing_data': # Robustness test (missing value)
#         for i in range(len(x_train)):
#             for j in range(x_train.shape[2]):
#                 mask = np.random.random(x_train[i][:, j].shape) >= args.ratio
#                 x_train[i][:, j] = x_train[i][:, j] * mask
#         for i in range(len(x_test)):
#             for j in range(x_test.shape[2]):
#                 mask = np.random.random(x_test[i][:, j].shape) >= args.ratio
#                 x_test[i][:, j] = x_test[i][:, j] * mask

#     args.num_labels = max(y_train) + 1
#     summary = [0 for i in range(args.num_labels)]
#     for i in y_train:
#         summary[i] += 1
#     args.log("Label num cnt: "+ str(summary))
#     args.log("Training size: " + str(len(y_train)))
#     args.log("Testing size: " + str(len(y_test)))
#     return list(x_train), y_train, list(x_test), y_test

In [ ]:
# xtrain, ytrain, xtest, ytest = read_data(args, config)

## Our pre-processing

In [51]:
from sliding_window import sliding_window
import pickle as cp
from tensorflow.keras.utils import to_categorical


In [52]:
def normalize(data):
    mean = data.mean()
    std =data.std()
    norm = (data - mean)/std
    return norm

In [53]:
# Number of Sensor Channels used in the OPPORTUNITY dataset.
NB_SENSOR_CHANNELS = 113

# Number of classes in which data is classified (or to be classified).
NUM_CLASSES = 5

# Length of the sliding window used to segmenting the time-series-data.
SLIDING_WINDOW_LENGTH = 24

# Steps of the sliding window used in segmenting the data.
SLIDING_WINDOW_STEP = 12

act_labels_txt = ['std', 'wlk', 'sit', 'lie', 'null']

In [54]:
def load_dataset(filename):

    with open(filename, 'rb') as f:
        data = cp.load(f,encoding='latin1')

    X_train, y_train = data[0]
    X_test, y_test = data[1]

    X_train = normalize(X_train)
    X_test = normalize(X_test)

    print(" ..from file {}".format(filename))
    print(" ..reading instances: train {0}, test {1}".format(X_train.shape, X_test.shape))

    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # The targets are casted to int8 for GPU compatibility.
    y_train = y_train.astype(np.uint8)
    y_test = y_test.astype(np.uint8)

    return X_train, y_train, X_test, y_test

print("Loading Data...")
X_train, y_train, X_test, y_test = load_dataset('......./oppChallenge_gestures.data')



assert NB_SENSOR_CHANNELS == X_train.shape[1]
def opp_sliding_window(data_x, data_y, ws, ss):
    data_x = sliding_window(data_x,(ws,data_x.shape[1]),(ss,1))
    data_y = np.asarray([[i[-1]] for i in sliding_window(data_y,ws,ss)])
    return data_x.astype(np.float32), data_y.reshape(len(data_y)).astype(np.uint8)

# Sensor data is segmented using a sliding window mechanism
X_test, y_test = opp_sliding_window(X_test, y_test, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (testing): inputs {0}, targets {1}".format(X_test.shape, y_test.shape))

# Data is reshaped since the input of the network is a 4 dimension tensor
X_test = X_test.reshape((-1, SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
# X_test = np.transpose(X_test, (0, 2, 1))
# X_test= X_test.reshape(X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]) # convert list to numpy array

Loading Data...
 ..from file .../oppChallenge_gestures.data
 ..reading instances: train (557963, 113), test (118750, 113)
 ..after sliding window (testing): inputs (9894, 24, 113), targets (9894,)


In [55]:
X_train, y_train = opp_sliding_window(X_train, y_train, SLIDING_WINDOW_LENGTH, SLIDING_WINDOW_STEP)
print(" ..after sliding window (training): inputs {0}, targets {1}".format(X_train.shape, y_train.shape))
X_train = X_train.reshape((-1,SLIDING_WINDOW_LENGTH, NB_SENSOR_CHANNELS))
# X_train = np.transpose(X_train, (0, 2, 1))
# X_train= X_train.reshape(X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]) # convert list to numpy array

X_train.shape


 ..after sliding window (training): inputs (46495, 24, 113), targets (46495,)


(46495, 24, 113)

In [56]:
# X_train = list(X_train)
# X_test = list(X_test)
xtrain, ytrain, xtest, ytest = X_train, y_train, X_test, y_test

Create a copy of xtest for evaluation


In [57]:
X_test = xtest

In [58]:
xtrain = corrupt3(xtrain)
xtest = corrupt3(xtest)

46495
24
113
9894
24
113


In [59]:
print(config.window_list)
print(config.stride_list)
print(config.k_list)
print(config.layer_num)
print(config.hidden_channel)
# Adjust model parameters based on our preprocessing
config.window_list = [7,16]
config.stride_list = [3,8]
config.k_list = [3,8]

[7, 16]
[3, 8]
[3, 8]
1
48


In [60]:
def test(model, xtest, ytest):
    #print(type(xtest))

    y_pred = []
    y_true = []
    with torch.no_grad():
        model.eval()
        for i in range(0, len(xtest), args.batch_size):
            if i + args.batch_size <= len(xtest):
               
                x = torch.Tensor(xtest[i: i+args.batch_size]).cuda()
               # x_new = float(x.item())
                # print(type(ytest[i: i+args.batch_size]))
                y_true += list(ytest[i: i+args.batch_size])
            else:
                x = torch.Tensor(xtest[i:]).cuda()
                y_true += list(ytest[i:])
            out = model(x)
            pred = torch.argmax(out, dim = -1)
            y_pred += pred.cpu().tolist()

    log("Accuracy : " + str(accuracy_score(y_true, y_pred)) +
        "\nWeighted F1 : " + str(f1_score(y_true, y_pred, labels=list(range(args.num_labels)),average='weighted')) )

In [ ]:
# random.seed(args.seed)
# random.shuffle(xtrain)
# random.seed(args.seed)
# random.shuffle(ytrain)

In [61]:
if args.model == 'UniTS':
    model = UniTS(input_size = args.input_size, sensor_num = args.input_channel, layer_num = config.layer_num,
    window_list = config.window_list,  stride_list = config.stride_list, k_list = config.k_list,
    out_dim = args.num_labels, hidden_channel = config.hidden_channel).cuda()

    optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
# elif args.model == 'static':
#     model = static_UniTS(input_size = args.input_size, sensor_num = args.input_channel, layer_num = config.layer_num,
#     window_list = config.window_list, stride_list = config.stride_list, k_list = config.k_list,
#     out_dim = args.num_labels, hidden_channel = config.hidden_channel).cuda()

#     optimizer = torch.optim.Adam(model.parameters(), lr = args.lr)
# elif args.model == 'THAT':
#     args.hlayers = 5
#     args.vlayers = 1
#     args.vheads = 16
#     args.K = 10
#     args.sample = 4
#     model = HARTrans(args).cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# elif args.model == 'RFNet':
#     model = RFNet(num_classes = args.num_labels, input_channel = args.input_channel, win_len = args.input_size).cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
# elif args.model == 'ResNet':
#     model = ResNet(input_size = args.input_size, input_channel = args.input_channel, num_label = args.num_labels).cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)        

# elif args.model == 'MaDNN':
#     model = MaDNN(input_size = args.input_size, input_channel = args.input_channel, num_label = args.num_labels).cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)   

# elif args.model == 'MaCNN':
#     model = MaCNN(input_size = args.input_size, input_channel = args.input_channel, num_label = args.num_labels, 
#         sensor_num = int(args.input_channel / args.SENSOR_AXIS)).cuda()
#     optimizer = torch.optim.SGD(model.parameters(), lr=args.lr)
# elif args.model == 'LaxCat':
#     model = LaxCat(input_size = args.input_size, input_channel = args.input_channel, num_label = args.num_labels,
#         hidden_dim = 64, kernel_size = 32, stride = 8).cuda()
#     optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

total_params = sum(p.numel() for p in model.parameters())
log('Total parameters: ' + str(total_params))

if args.test_only:
    if os.path.exists(args.model_save_path):
        model.load_state_dict(torch.load(args.model_save_path))
       
        test(model, xtest, ytest)

    else:
        log("Model state dict not found!")
    # return


Total parameters: 512927


In [64]:
model.load_state_dict(torch.load(args.model_save_path))

<All keys matched successfully>

In [19]:
torch.load(args.model_save_path).keys()

odict_keys(['ts_encoders.0.FIC.conv.weight', 'ts_encoders.0.RPC.weight', 'ts_encoders.0.RPC.bias', 'ts_encoders.1.FIC.conv.weight', 'ts_encoders.1.RPC.weight', 'ts_encoders.1.RPC.bias', 'multi_channel_fusion.0.weight', 'multi_channel_fusion.0.bias', 'multi_channel_fusion.1.weight', 'multi_channel_fusion.1.bias', 'conv_branches.0.0.conv1.0.weight', 'conv_branches.0.0.conv1.0.bias', 'conv_branches.0.0.bn1.0.weight', 'conv_branches.0.0.bn1.0.bias', 'conv_branches.0.0.bn1.0.running_mean', 'conv_branches.0.0.bn1.0.running_var', 'conv_branches.0.0.bn1.0.num_batches_tracked', 'conv_branches.0.0.conv2.0.weight', 'conv_branches.0.0.conv2.0.bias', 'conv_branches.0.0.bn2.0.weight', 'conv_branches.0.0.bn2.0.bias', 'conv_branches.0.0.bn2.0.running_mean', 'conv_branches.0.0.bn2.0.running_var', 'conv_branches.0.0.bn2.0.num_batches_tracked', 'conv_branches.0.1.weight', 'conv_branches.0.1.bias', 'conv_branches.0.3.conv1.0.weight', 'conv_branches.0.3.conv1.0.bias', 'conv_branches.0.3.bn1.0.weight', 'con

In [62]:
loss_func = nn.CrossEntropyLoss()
try:
    for ep in range(1, 1+args.epochs):
        model.train()
        epoch_loss = 0
        log("Training epoch : " + str(ep))
        for i in range(0, len(xtrain), args.batch_size):
            if i + args.batch_size <= len(xtrain):
                x = torch.Tensor(xtrain[i: i+args.batch_size]).cuda()

                y = torch.LongTensor(ytrain[i: i+args.batch_size]).cuda()  
            else:
                x = torch.Tensor(xtrain[i:]).cuda()
                y = torch.LongTensor(ytrain[i:]).cuda()                      
            out = model(x)
            loss = loss_func(out, y)
            epoch_loss += loss.cpu().item()

            optimizer.zero_grad()           
            loss.backward()
            optimizer.step()

        log("Training loss : " + str(epoch_loss / (i / args.batch_size + 1)))
     
        test(model, xtest, ytest)

        log("----------------------------")


except KeyboardInterrupt:
    print('Exiting from training early')
    test(model, xtest, ytest)
if args.save:
    torch.save(model.state_dict(), args.model_save_path)

Training epoch : 1
Training loss : 1.1094472211320594
Accuracy : 0.7290276935516474
Weighted F1 : 0.6599292126196044
----------------------------
Training epoch : 2
Training loss : 0.8336756201627494
Accuracy : 0.7375176874873661
Weighted F1 : 0.690538210373143
----------------------------
Training epoch : 3
Training loss : 0.6319536424691609
Accuracy : 0.7552051748534465
Weighted F1 : 0.7228710654150127
----------------------------
Training epoch : 4
Training loss : 0.502492505872553
Accuracy : 0.7624823125126339
Weighted F1 : 0.7372415503426201
----------------------------
Training epoch : 5
Training loss : 0.4005906425846902
Accuracy : 0.7603598140287042
Weighted F1 : 0.7413536603975446
----------------------------
Training epoch : 6
Training loss : 0.32053625598636953
Accuracy : 0.757630887406509
Weighted F1 : 0.7359470286449848
----------------------------
Training epoch : 7
Training loss : 0.242390499139612
Accuracy : 0.7554073175662017
Weighted F1 : 0.732374367061669
-----------

In [63]:
torch.save(model.state_dict(), args.model_save_path)

Evaluation

In [74]:
#xtest_eval = corrupt1(X_test,sigma=0.05)
xtest_eval = corrupt3(X_test,sigma=[0.2,8, 10])


test(model, xtest_eval, ytest)

9894
24
113
Accuracy : 0.7473216090559935
Weighted F1 : 0.7267344819479478
